In [30]:
import numpy as np


test_data_raw =(
"""O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""
)


def parse_data(data):
    return np.array([list(line) for line in data.splitlines()])


def print_data(data):
    for line in data:
        print(''.join(line))


test_data = parse_data(test_data_raw)
print_data(test_data)

O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....


In [34]:
def tilt_north(data):
    data = data.copy()
    for col_idx in range(data.shape[1]):
        col = "".join(data[:, col_idx])
        new_col = "#".join([
            "O" * subcol.count("O") + "." * subcol.count(".")
            for subcol in col.split("#")
        ])
        data[:, col_idx] = list(new_col)
    return data


print_data(tilt_north(test_data))

OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....


In [39]:
def get_total_load(tilted_data):
    xs, _ = np.where(tilted_data == "O")
    return (tilted_data.shape[0] - xs).sum()


def part1(data):
    return get_total_load(tilt_north(data))


part1(test_data)

136

In [38]:
with open("input.txt") as f:
    data = parse_data(f.read())

print_data(data[:5, :5])

#....
O....
.....
.OO..
#....


In [40]:
part1(data)

108144